# AWS: Exposing a service

## Connecting Application With Services

Before discussing the Kubernetes approach to networking, it is worthwhile to contrast it with the “normal” way networking works with Docker.

By default, Docker uses host-private networking, so containers can talk to other containers only if they are on the same machine. In order for Docker containers to communicate across nodes, there must be allocated ports on the machine’s own IP address, which are then forwarded or proxied to the containers. This obviously means that containers must either coordinate which ports they use very carefully or ports must be allocated dynamically.

Coordinating ports across multiple developers is very difficult to do at scale and exposes users to cluster-level issues outside of their control. Kubernetes assumes that pods can communicate with other pods, regardless of which host they land on. We give every pod its own cluster-private-IP address so you do not need to explicitly create links between pods or map container ports to host ports. This means that containers within a Pod can all reach each other’s ports on localhost, and all pods in a cluster can see each other without NAT.

### Exposing pods to the cluster

Create a nginx deployment, and note that it has a container port specification:

```yml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: my-nginx
spec:
  selector:
    matchLabels:
      run: my-nginx
  replicas: 2
  template:
    metadata:
      labels:
        run: my-nginx
    spec:
      containers:
      - name: my-nginx
        image: nginx
        ports:
        - containerPort: 80
```

This makes it accessible from any node in your cluster. Check the nodes the Pod is running on:

    # create the nginx deployment with 2 replicas
    kubectl apply -f ex01-run-my-nginx.yaml

    kubectl get pods -o wide

Check your pods’ IPs:

    kubectl get pods -o yaml | grep 'podIP:'

### Creating a Service

So we have pods running nginx in a flat, cluster wide, address space. In theory, you could talk to these pods directly, but what happens when a node dies? The pods die with it, and the Deployment will create new ones, with different IPs. This is the problem a Service solves.

A Kubernetes Service is an abstraction which defines a logical set of Pods running somewhere in your cluster, that all provide the same functionality. When created, each Service is assigned a unique IP address (also called clusterIP). This address is tied to the lifespan of the Service, and will not change while the Service is alive. Pods can be configured to talk to the Service, and know that communication to the Service will be automatically load-balanced out to some pod that is a member of the Service.

You can create a Service for your 2 nginx replicas with kubectl expose:

    kubectl expose deployment my-nginx

This specification will create a Service which targets TCP port 80 on any Pod with the run: my-nginx label, and expose it on an abstracted Service port (targetPort: is the port the container accepts traffic on, port: is the abstracted Service port, which can be any port other pods use to access the Service). View Service API object to see the list of supported fields in service definition. Check your Service:

    kubectl get svc my-nginx

As mentioned previously, a Service is backed by a group of Pods. These Pods are exposed through endpoints. The Service’s selector will be evaluated continuously and the results will be POSTed to an Endpoints object also named my-nginx. When a Pod dies, it is automatically removed from the endpoints, and new Pods matching the Service’s selector will automatically get added to the endpoints. Check the endpoints, and note that the IPs are the same as the Pods created in the first step:

    kubectl describe svc my-nginx

You should now be able to curl the nginx Service on CLUSTER-IP: PORT from any pods in your cluster.

    kubectl exec my-nginx-75897978cd-kq5gp -- curl 10.100.173.156

> The Service IP is completely virtual, it never hits the wire.

Let’s try that by :

Setting a variable called MyClusterIP with the my-nginx Service IP.

    # Create a variable set with the my-nginx service IP
    export MyClusterIP=$(kubectl get svc my-nginx -ojsonpath='{.spec.clusterIP}')

Creating a new deployment called load-generator (with the MyClusterIP variable also set inside the container) and get an interactive shell on a pod + container.

    # Create a new deployment and allocate a TTY for the container in the pod
    kubectl run -i --tty load-generator --env="MyClusterIP=${MyClusterIP}" --image=busybox  /bin/sh

> kubectl run --generator=deployment/apps.v1 is DEPRECATED and will be removed in a future version. Use kubectl run --generator=run-pod/v1 or kubectl create instead.

Connecting to the nginx welcome page using the ClusterIP.

    wget -q -O - ${MyClusterIP} | grep '<title>'

Type exit to log out of the container:

    exit

Druge možnosti kako preusmerjati promet med različnimi service-i (E-W traffic):
- service mash

## Accessing the Service

Kubernetes supports 2 primary modes of finding a Service: environment variables and DNS.

The former works out of the box while the latter requires the CoreDNS cluster add-on (automatically installed when creating the EKS cluster).



### Environment Variables

When a Pod runs on a Node, the kubelet adds a set of environment variables for each active Service. This introduces an ordering problem. To see why, inspect the environment of your running nginx Pods (your Pod name will be different): Let’s view the pods again:

    kubectl  get pods -o wide

Now let’s inspect the environment of one of your running nginx Pods:

    kubectl exec my-nginx-75897978cd-kq5gp -- printenv | grep SERVICE

    KUBERNETES_SERVICE_PORT=443
    KUBERNETES_SERVICE_HOST=10.100.0.1
    KUBERNETES_SERVICE_PORT_HTTPS=443

Note there’s no mention of your Service. This is because you created the replicas before the Service.

Another disadvantage of doing this is that the scheduler might put both Pods on the same machine, which will take your entire Service down if it dies. We can do this the right way by killing the 2 Pods and waiting for the Deployment to recreate them. This time around the Service exists before the replicas. This will give you scheduler-level Service spreading of your Pods (provided all your nodes have equal capacity), as well as the right environment variables:
    

    kubectl rollout restart deployment my-nginx

    kubectl get pods  -o wide

> You may notice that the pods have different names, since they are destroyed and recreated.

Now let’s inspect the environment of one of your running nginx Pods one more time:

    kubectl exec my-nginx-7c48ddbdcf-mdj5t -- printenv | grep SERVICE

    MY_NGINX_SERVICE_PORT=80
    KUBERNETES_SERVICE_PORT=443
    KUBERNETES_SERVICE_PORT_HTTPS=443
    KUBERNETES_SERVICE_HOST=10.100.0.1
    MY_NGINX_SERVICE_HOST=10.100.173.156

We now have an environment variable referencing the nginx Service IP called MY_NGINX_SERVICE_HOST.

### DNS


Kubernetes offers a DNS cluster add-on Service that automatically assigns dns names to other Services. You can check if it’s running on your cluster:

To check if your cluster is already running CoreDNS, use the following command.

    kubectl get pod -n kube-system -l k8s-app=kube-dns



> The service for CoreDNS is still called kube-dns for backward compatibility.

If it isn’t running, you can enable it. The rest of this section will assume you have a Service with a long lived IP (my-nginx), and a DNS server that has assigned a name to that IP (the CoreDNS cluster addon), so you can talk to the Service from any pod in your cluster using standard methods (e.g. gethostbyname). Let’s run another curl application to test this:

    kubectl run curl --generator=run-pod/v1 --image=radial/busyboxplus:curl -i --tty 

Then, hit enter and run.

    nslookup my-nginx

## Exposing the Service

For some parts of your applications you may want to expose a Service onto an external IP address. Kubernetes supports two ways of doing this: NodePort and LoadBalancer.

    kubectl  get svc my-nginx

Currently the Service does not have an External IP, so let’s now patch the Service to use a cloud load balancer, by updating the type of the my-nginx Service from ClusterIP to LoadBalancer:

    kubectl patch svc my-nginx -p '{"spec": {"type": "LoadBalancer"}}'

We can check for the changes:

    kubectl  get svc my-nginx

> The Load Balancer can take a couple of minutes in being available on the DNS.

    export loadbalancer=$(kubectl get svc my-nginx -o jsonpath='{.status.loadBalancer.ingress[*].hostname}')

    curl -k -s http://${loadbalancer} | grep title

If the Load Balancer name is too long to fit in the standard kubectl get svc output, you’ll need to do kubectl describe service my-nginx to see it. You’ll see something like this:

    kubectl describe service my-nginx | grep Ingress

## Ingress

What is Ingress? Ingress, added in Kubernetes v1.1, exposes HTTP and HTTPS routes from outside the cluster to services within the cluster. Traffic routing is controlled by rules defined on the Ingress resource.



    internet
          |
      [ Ingress ]
      --|-----|--
      [ Services ]

An Ingress can be configured to give services externally-reachable URLs, load balance traffic, terminate SSL, and offer name based virtual hosting. An Ingress controller is responsible for fulfilling the Ingress, usually with a load balancer, though it may also configure your edge router or additional frontends to help handle the traffic.

An Ingress does not expose arbitrary ports or protocols. Exposing services other than HTTP and HTTPS to the internet typically uses a service of type NodePort or LoadBalancer.

You must have an ingress controller to satisfy an Ingress. Only creating an Ingress resource has no effect.

You may need to deploy an Ingress controller such as AWS ALB Ingress Controller. You can choose from a number of Ingress controllers.

Ideally, all Ingress controllers should fit the reference specification. In reality, the various Ingress controllers operate slightly differently.

### The Ingress Resource

A minimal ingress resource example for ingress-nginx:

    apiVersion: extensions/v1beta1
    kind: Ingress
    metadata:
      name: test-ingress
      annotations:
        nginx.ingress.kubernetes.io/rewrite-target: /
    spec:
      rules:
      - http:
          paths:
          - path: /testpath
            backend:
              serviceName: test
              servicePort: 80

<p>As with all other Kubernetes resources, an Ingress needs <code>apiVersion</code><span class="copy-to-clipboard" title="Copy to clipboard"></span>, <code>kind</code>, and <code>metadata</code><span class="copy-to-clipboard" title="Copy to clipboard"></span> fields. The name of an Ingress object must be a valid <a href="https://kubernetes.io/docs/concepts/overview/working-with-objects/names#dns-subdomain-names" target="_blank" class="highlight">DNS subdomain name</a>. For general information about working with config files, see <a href="https://kubernetes.io/docs/tasks/run-application/run-stateless-application-deployment/" target="_blank" class="highlight">deploying applications</a>, <a href="https://kubernetes.io/docs/tasks/configure-pod-container/configure-pod-configmap/" target="_blank" class="highlight">configuring containers</a>, <a href="https://kubernetes.io/docs/concepts/cluster-administration/manage-deployment/" target="_blank" class="highlight">managing resources</a>. Ingress frequently uses annotations to configure some options depending on the Ingress controller, an example of which is the <a href="https://github.com/kubernetes/ingress-nginx/blob/master/docs/examples/rewrite/README.md" target="_blank" class="highlight">rewrite-target annotation</a>. Different <a href="https://kubernetes.io/docs/concepts/services-networking/ingress-controllers" target="_blank" class="highlight">Ingress controller</a> support different annotations. Review the documentation for your choice of Ingress controller to learn which annotations are supported.</p>

The Ingress spec has all the information needed to configure a load balancer or proxy server. Most importantly, it contains a list of rules matched against all incoming requests. Ingress resource only supports rules for directing HTTP traffic.

### Ingress rules


Each http rule contains the following information:

<ul>
<li>An optional host. In this example, no host is specified, so the rule applies to all inbound HTTP traffic through the IP address specified. If a host is provided (for example, foo.bar.com), the rules apply to that host.</li>
<li>A list of paths (for example, <code>/testpath</code><span class="copy-to-clipboard" title="Copy to clipboard"></span>), each of which has an associated backend defined with a <code>serviceName</code><span class="copy-to-clipboard" title="Copy to clipboard"></span> and <code>servicePort</code><span class="copy-to-clipboard" title="Copy to clipboard"></span>. Both the host and path must match the content of an incoming request before the load balancer will direct traffic to the referenced service.</li>
<li>A backend is a combination of service and port names as described in the <a href="https://kubernetes.io/docs/concepts/services-networking/service/" target="_blank" class="highlight">Services doc</a>. HTTP (and HTTPS) requests to the Ingress matching the host and path of the rule will be sent to the listed backend.</li>
</ul>

A default backend is often configured in an Ingress controller that will service any requests that do not match a path in the spec.

### Default Backend

An Ingress with no rules sends all traffic to a single default backend. The default backend is typically a configuration option of the Ingress controller and is not specified in your Ingress resources.

If none of the hosts or paths match the HTTP request in the Ingress objects, the traffic is routed to your default backend.

## Ingress Controllers

In order for the Ingress resource to work, the cluster must have an ingress controller running.

Unlike other types of controllers which run as part of the kube-controller-manager binary, Ingress controllers are not started automatically with a cluster.

### AWS ALB Ingress Controller

<p><a href="https://docs.aws.amazon.com/elasticloadbalancing/latest/application/introduction.html" target="_blank" class="highlight">AWS Elastic Load Balancing Application Load Balancer (ALB)</a> is a popular AWS service that load balances incoming traffic at the application layer (layer 7) across multiple targets, such as Amazon EC2 instances, in multiple Availability Zones. ALB supports multiple features including host or path based routing, TLS (Transport Layer Security) termination, WebSockets, HTTP/2, AWS WAF (Web Application Firewall) integration, integrated access logs, and health checks.</p>

<p>The open source <a href="https://github.com/kubernetes-sigs/aws-alb-ingress-controller" target="_blank" class="highlight">AWS ALB Ingress controller</a> triggers the creation of an ALB and the necessary supporting AWS resources whenever a Kubernetes user declares an Ingress resource in the cluster. The Ingress resource uses the ALB to route HTTP(S) traffic to different endpoints within the cluster. The AWS ALB Ingress controller works on any Kubernetes cluster including Amazon Elastic Kubernetes Service (Amazon EKS).</p>

### Deploy AWS ALB Ingress controller

First, create an IAM OIDC provider and associate it with your cluster:

    eksctl utils associate-iam-oidc-provider --cluster=icta-plume --approve

> Managment console: https://docs.aws.amazon.com/eks/latest/userguide/enable-iam-roles-for-service-accounts.html

Next, deploy the relevant RBAC roles and role bindings as required by the AWS ALB Ingress controller:

    kubectl apply -f ex-02-rbac-role.yaml

Next, create an IAM policy named `ALBIngressControllerIAMPolicy` to allow the ALB Ingress controller to make AWS API calls on your behalf and save the `Policy.Arn` into a new variable called PolicyARN:

> Delamo z admin računom

    #create the policy
    aws iam create-policy   --policy-name ALBIngressControllerIAMPolicy   --policy-document file://iam-policy.json  --profile administrator

    #get the policy ARN
    export PolicyARN=$(aws iam list-policies --query 'Policies[?PolicyName==`ALBIngressControllerIAMPolicy`].Arn' --profile administrator --output text)

Next, create a Kubernetes service account and an IAM role (for the pod running the AWS ALB Ingress controller):

    eksctl create iamserviceaccount \
            --cluster=eksworkshop-eksctl \
            --namespace=kube-system \
            --name=alb-ingress-controller \
            --attach-policy-arn=$PolicyARN \
            --override-existing-serviceaccounts \
            --approve

- https://docs.aws.amazon.com/eks/latest/userguide/create-service-account-iam-policy-and-role.html
- https://docs.aws.amazon.com/eks/latest/userguide/specify-service-account-role.html 

    echo 'export ALB_INGRESS_VERSION="v1.1.8"' >>  ~/.bash_profile
    .  ~/.bash_profile

Then, deploy AWS ALB Ingress controller

    # We dynamically replace the cluster-name by the name of our cluster before applying the YAML file
    curl -sS "https://raw.githubusercontent.com/kubernetes-sigs/aws-alb-ingress-controller/${ALB_INGRESS_VERSION}/docs/examples/alb-ingress-controller.yaml" \
        | sed 's/# - --cluster-name=devCluster/- --cluster-name=eksworkshop-eksctl/g' \
        | kubectl apply -f -

Verify that the deployment was successful and the controller started:

    kubectl logs -n kube-system $(kubectl get po -n kube-system | egrep -o alb-ingress[a-zA-Z0-9-]+)

    kubectl apply -f https://raw.githubusercontent.com/kubernetes-sigs/aws-alb-ingress-controller/${ALB_INGRESS_VERSION}/docs/examples/2048/2048-namespace.yaml
    kubectl apply -f https://raw.githubusercontent.com/kubernetes-sigs/aws-alb-ingress-controller/${ALB_INGRESS_VERSION}/docs/examples/2048/2048-deployment.yaml
    kubectl apply -f https://raw.githubusercontent.com/kubernetes-sigs/aws-alb-ingress-controller/${ALB_INGRESS_VERSION}/docs/examples/2048/2048-service.yaml

https://www.eksworkshop.com/beginner/130_exposing-service/